[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MarcCote/TextWorld/blob/msr_summit_2021/notebooks/Building%20a%20simple%20agent.ipynb)

# Building a simple agent with TextWorld

This tutorial outlines the steps to build an agent that learns how to play __choice-based__ text-based games generated with TextWorld.

In [1]:
IS_COLAB = False
try:
    import google.colab, torch, sys
    print(torch.__version__)
    if not torch.cuda.is_available():
        print("Change runtime type to include a GPU.")
    IS_COLAB = True
except:
    pass

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd drive/My Drive
    %cd trl
    
print(IS_COLAB)

False


### Prerequisite
Install TextWorld as described in the [README.md](https://github.com/microsoft/TextWorld#readme). Most of the time, a simple `pip install` should work.

In [2]:
if IS_COLAB:
    !pip install textworld

In [3]:
if IS_COLAB:
    !pip install torch torchvision torchaudio
    !pip install transformers

and [PyTorch](https://pytorch.org/) (tested with both v1.8.2 and v.1.9.0).

In [4]:
import torch
print(torch.__version__)

1.12.1+cu116


**[Optional]** Download all data beforehand. Otherwise, they are going to be generated as needed (slower).

In [5]:
%pwd

'/home/ubuntu/trl'

In [6]:
!wget https://aka.ms/textworld/notebooks/data.zip
!unzip -nq data.zip && rm -f data.zip

--2022-08-25 08:05:52--  https://aka.ms/textworld/notebooks/data.zip
Resolving aka.ms (aka.ms)... 104.87.239.185
Connecting to aka.ms (aka.ms)|104.87.239.185|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://textworld.blob.core.windows.net/$web/notebooks/data.zip [following]
--2022-08-25 08:05:52--  https://textworld.blob.core.windows.net/$web/notebooks/data.zip
Resolving textworld.blob.core.windows.net (textworld.blob.core.windows.net)... 52.239.172.164
Connecting to textworld.blob.core.windows.net (textworld.blob.core.windows.net)|52.239.172.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26926729 (26M) [application/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]  25.68M  27.1MB/s    in 0.9s    

2022-08-25 08:05:53 (27.1 MB/s) - ‘data.zip’ saved [26926729/26926729]



## Learning challenges
Training an agent such that it can learn how to play text-based games is not trivial. Among other challenges, we have to deal with

1. a combinatorial action space (that grows w.r.t. vocabulary)
2. a really sparse reward signal.

To ease the learning process, we will be requesting additional information alongside the game's narrative (as covered in [Playing TextWorld generated games with OpenAI Gym](Playing%20TextWorld%20generated%20games%20with%20OpenAI%20Gym.ipynb#Interact-with-the-game)). More specifically, we will request the following information:

- __Description__:
For every game state, we will get the output of the `look` command which describes the current location;

- __Inventory__:
For every game state, we will get the output of the `inventory` command which describes the player's inventory;

- __Admissible commands__:
For every game state, we will get the list of commands guaranteed to be understood by the game interpreter;

- __Intermediate reward__:
For every game state, we will get an intermediate reward which can either be:
  - __-1__: last action needs to be undone before resuming the quest
  -  __0__: last action didn't affect the quest
  -  __1__: last action brought us closer to completing the quest

- __Entities__:
For every game, we will get a list of entity names that the agent can interact with.


## Simple test games
We can use TextWorld to generate a few simple games with the following handcrafted world
```
                     Bathroom
                        +
                        |
                        +
    Bedroom +-(d1)-+ Kitchen +--(d2)--+ Backyard
      (P)               +                  +
                        |                  |
                        +                  +
                   Living Room           Garden
```
where the goal is always to retrieve a hidden food item and put it on the stove which is located in the kitchen. One can lose the game if it eats the food item instead of putting it on the stove!

Using `tw-make tw-simple ...`, we are going to generate the following 7 games:

| gamefile | description |
| :------- | :---------- |
| `games/rewardsDense_goalDetailed.z8` | dense reward + detailed instructions |
| `games/rewardsBalanced_goalDetailed.z8` | balanced rewards + detailed instructions |
| `games/rewardsSparse_goalDetailed.z8` | sparse rewards + detailed instructions |
| | |
| `games/rewardsDense_goalBrief.z8` | dense rewards + no instructions but the goal is mentionned |
| `games/rewardsBalanced_goalBrief.z8` | balanced rewards + no instructions but the goal is mentionned |
| `games/rewardsSparse_goalBrief.z8` | sparse rewards + no instructions but the goal is mentionned |
| | |
| `games/rewardsSparse_goalNone.z8` | sparse rewards + no instructions/goal<br>_Hint: there's an hidden note in the game that describes the goal!_ |

In [7]:
# You can skip this if you already downloaded the games in the prequisite section.
if False:
  # Same as !make_games.sh
  !tw-make tw-simple --rewards dense    --goal detailed --seed 18 --test --silent -f --output games/tw-rewardsDense_goalDetailed.z8
  !tw-make tw-simple --rewards balanced --goal detailed --seed 18 --test --silent -f --output games/tw-rewardsBalanced_goalDetailed.z8
  !tw-make tw-simple --rewards sparse   --goal detailed --seed 18 --test --silent -f --output games/tw-rewardsSparse_goalDetailed.z8
  !tw-make tw-simple --rewards dense    --goal brief    --seed 18 --test --silent -f --output games/tw-rewardsDense_goalBrief.z8
  !tw-make tw-simple --rewards balanced --goal brief    --seed 18 --test --silent -f --output games/tw-rewardsBalanced_goalBrief.z8
  !tw-make tw-simple --rewards sparse   --goal brief    --seed 18 --test --silent -f --output games/tw-rewardsSparse_goalBrief.z8
  !tw-make tw-simple --rewards sparse   --goal none     --seed 18 --test --silent -f --output games/tw-rewardsSparse_goalNone.z8

## Building the random baseline
Let's start with building an agent that simply selects an admissible command at random.

In [8]:
from typing import Mapping, Any

import numpy as np

import textworld.gym


class RandomAgent(textworld.gym.Agent):
    """ Agent that randomly selects a command from the admissible ones. """
    def __init__(self, seed=1234):
        self.seed = seed
        self.rng = np.random.RandomState(self.seed)

    @property
    def infos_to_request(self) -> textworld.EnvInfos:
        return textworld.EnvInfos(admissible_commands=True)
    
    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:
        return self.rng.choice(infos["admissible_commands"])


In [9]:
class HumanAgent(textworld.gym.Agent):
    """ Agent that randomly selects a command from the admissible ones. """
    def __init__(self, seed=1234):
        self.seed = seed
        self.rng = np.random.RandomState(self.seed)

    @property
    def infos_to_request(self) -> textworld.EnvInfos:
        return textworld.EnvInfos(admissible_commands=True)
    
    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:
        print(obs)
        return input()

## Play function
Let's write a simple play function that we will use to evaluate our agent on a given game.

In [10]:
import os
from glob import glob

import gym
import textworld.gym

import torch


def play(agent, path, max_step=100, nb_episodes=10, verbose=True):
    torch.manual_seed(20211021)  # For reproducibility when using action sampling.

    infos_to_request = agent.infos_to_request
    infos_to_request.max_score = True  # Needed to normalize the scores.
    
    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.z8"))
        
    print(gamefiles)
    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=infos_to_request,
                                          max_episode_steps=max_step)
    print(env_id)
    env = gym.make(env_id)  # Create a Gym environment to play the text game.
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path), end="")
        else:
            print(os.path.basename(path), end="")
        
    # Collect some statistics: nb_steps, final reward.
    avg_moves, avg_scores, avg_norm_scores = [], [], []
    for no_episode in range(nb_episodes):
        obs, infos = env.reset()  # Start new episode.

        score = 0
        done = False
        nb_moves = 0
        while not done:
            command = agent.act(obs, score, done, infos)
            obs, score, done, infos = env.step(command)
            if hasattr(agent, 'reportScore'):
                agent.reportScore(score, done, infos)
            nb_moves += 1
        
        agent.act(obs, score, done, infos)  # Let the agent know the game is done.
                
        if verbose:
            print(".", end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score / infos["max_score"])

    env.close()
    if verbose:
        if os.path.isdir(path):
            msg = "  \tavg. steps: {:5.1f}; avg. normalized score: {:4.1f} / {}."
            print(msg.format(np.mean(avg_moves), np.mean(avg_norm_scores), 1))
        else:
            msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
            print(msg.format(np.mean(avg_moves), np.mean(avg_scores), infos["max_score"]))
    

#### Evaluate the random agent

In [11]:
%pwd
%ls

CONTRIBUTING.md  checkpoints/  requirements.txt    testing_games/
LICENSE          docs/         settings.ini        training_games/
MANIFEST.in      games/        setup.py            trl/
Makefile         gpt2.ipynb    test_trl.py
README.md        nbs/          test_trl_gptneo.py


In [12]:
import os
print(os.path.exists("./games/tw-rewardsDense_goalDetailed.z8"))

True


In [13]:
if False:
    # We report the score and steps averaged over 10 playthroughs.
    play(RandomAgent(), "./games/tw-rewardsDense_goalDetailed.z8")    # Dense rewards
    play(RandomAgent(), "./games/tw-rewardsBalanced_goalDetailed.z8") # Balanced rewards
    play(RandomAgent(), "./games/tw-rewardsSparse_goalDetailed.z8")   # Sparse rewards

In [14]:
if False:
    play(HumanAgent(), "./games/tw-rewardsDense_goalDetailed.z8")

## Neural agent

Now, let's create an agent that can learn to play text-based games. The agent will be trained to select a command from the list of admissible commands given the current game's narrative, inventory, and room description. Here is an overview of the architecture used for the agent: 

<div>
  <img src="https://raw.githubusercontent.com/MarcCote/TextWorld/msr_summit_2021/notebooks/figs/neural_agent.png" width="500"/>
</div>



### Code
Here's the implementation of that learning agent built with [PyTorch](https://pytorch.org/).

In [15]:
import re
from typing import List, Mapping, Any, Optional
from collections import defaultdict

import numpy as np

import textworld
import textworld.gym
from textworld import EnvInfos

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


class CommandScorer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CommandScorer, self).__init__()
        torch.manual_seed(42)  # For reproducibility
        self.embedding    = nn.Embedding(input_size, hidden_size)
        self.encoder_gru  = nn.GRU(hidden_size, hidden_size)
        self.cmd_encoder_gru  = nn.GRU(hidden_size, hidden_size)
        self.state_gru    = nn.GRU(hidden_size, hidden_size)
        self.hidden_size  = hidden_size
        self.state_hidden = torch.zeros(1, 1, hidden_size, device=device)
        self.critic       = nn.Linear(hidden_size, 1)
        self.att_cmd      = nn.Linear(hidden_size * 2, 1)

    def forward(self, obs, commands, **kwargs):
        input_length = obs.size(0)
        batch_size = obs.size(1)
        nb_cmds = commands.size(1)

        embedded = self.embedding(obs)
        encoder_output, encoder_hidden = self.encoder_gru(embedded)
        state_output, state_hidden = self.state_gru(encoder_hidden, self.state_hidden)
        self.state_hidden = state_hidden
        value = self.critic(state_output)

        # Attention network over the commands.
        cmds_embedding = self.embedding.forward(commands)
        _, cmds_encoding_last_states = self.cmd_encoder_gru.forward(cmds_embedding)  # 1 x cmds x hidden

        # Same observed state for all commands.
        cmd_selector_input = torch.stack([state_hidden] * nb_cmds, 2)  # 1 x batch x cmds x hidden

        # Same command choices for the whole batch.
        cmds_encoding_last_states = torch.stack([cmds_encoding_last_states] * batch_size, 1)  # 1 x batch x cmds x hidden

        # Concatenate the observed state and command encodings.
        cmd_selector_input = torch.cat([cmd_selector_input, cmds_encoding_last_states], dim=-1)

        # Compute one score per command.
        scores = F.relu(self.att_cmd(cmd_selector_input)).squeeze(-1)  # 1 x Batch x cmds

        probs = F.softmax(scores, dim=2)  # 1 x Batch x cmds
        index = probs[0].multinomial(num_samples=1).unsqueeze(0) # 1 x batch x indx
        return scores, index, value

    def reset_hidden(self, batch_size):
        self.state_hidden = torch.zeros(1, batch_size, self.hidden_size, device=device)


class NeuralAgent:
    """ Simple Neural Agent for playing TextWorld games. """
    MAX_VOCAB_SIZE = 1000
    UPDATE_FREQUENCY = 10
    LOG_FREQUENCY = 1000
    GAMMA = 0.9
    
    def __init__(self) -> None:
        self._initialized = False
        self._epsiode_has_started = False
        self.id2word = ["<PAD>", "<UNK>"]
        self.word2id = {w: i for i, w in enumerate(self.id2word)}
        
        self.model = CommandScorer(input_size=self.MAX_VOCAB_SIZE, hidden_size=128)
        if device == "cuda":
          self.model = self.model.cuda()
          
        self.optimizer = optim.Adam(self.model.parameters(), 0.00003)
        
        self.mode = "test"
    
    def train(self):
        self.mode = "train"
        self.stats = {"max": defaultdict(list), "mean": defaultdict(list)}
        self.transitions = []
        self.model.reset_hidden(1)
        self.last_score = 0
        self.no_train_step = 0
    
    def test(self):
        self.mode = "test"
        self.model.reset_hidden(1)
        
    @property
    def infos_to_request(self) -> EnvInfos:
        return EnvInfos(description=True, inventory=True, admissible_commands=True,
                        won=True, lost=True)
    
    def _get_word_id(self, word):
        if word not in self.word2id:
            if len(self.word2id) >= self.MAX_VOCAB_SIZE:
                return self.word2id["<UNK>"]
            
            self.id2word.append(word)
            self.word2id[word] = len(self.word2id)
            
        return self.word2id[word]
            
    def _tokenize(self, text):
        # Simple tokenizer: strip out all non-alphabetic characters.
        text = re.sub("[^a-zA-Z0-9\- ]", " ", text)
        word_ids = list(map(self._get_word_id, text.split()))
        return word_ids

    def _process(self, texts):
        texts = list(map(self._tokenize, texts))
        max_len = max(len(l) for l in texts)
        padded = np.ones((len(texts), max_len)) * self.word2id["<PAD>"]

        for i, text in enumerate(texts):
            padded[i, :len(text)] = text

        padded_tensor = torch.from_numpy(padded).type(torch.long).to(device)
        padded_tensor = padded_tensor.permute(1, 0) # Batch x Seq => Seq x Batch
        return padded_tensor
      
    def _discount_rewards(self, last_values):
        returns, advantages = [], []
        R = last_values.data
        for t in reversed(range(len(self.transitions))):
            rewards, _, _, values = self.transitions[t]
            R = rewards + self.GAMMA * R
            adv = R - values
            returns.append(R)
            advantages.append(adv)
            
        return returns[::-1], advantages[::-1]

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> Optional[str]:
        
        # Build agent's observation: feedback + look + inventory.
        input_ = "{}\n{}\n{}".format(obs, infos["description"], infos["inventory"])
        
        # Tokenize and pad the input and the commands to chose from.
        input_tensor = self._process([input_]).to(device)
        commands_tensor = self._process(infos["admissible_commands"]).to(device)
        
        # Get our next action and value prediction.
        outputs, indexes, values = self.model(input_tensor, commands_tensor)
        action = infos["admissible_commands"][indexes[0]]
        
        if self.mode == "test":
            if done:
                self.model.reset_hidden(1)
            return action
        
        self.no_train_step += 1
        
        if self.transitions:
            reward = score - self.last_score  # Reward is the gain/loss in score.
            self.last_score = score
            if infos["won"]:
                reward += 100
            if infos["lost"]:
                reward -= 100
                
            self.transitions[-1][0] = reward  # Update reward information.
        
        self.stats["max"]["score"].append(score)
        if self.no_train_step % self.UPDATE_FREQUENCY == 0:
            # Update model
            returns, advantages = self._discount_rewards(values)
            
            loss = 0
            for transition, ret, advantage in zip(self.transitions, returns, advantages):
                reward, indexes_, outputs_, values_ = transition
                
                advantage        = advantage.detach() # Block gradients flow here.
                probs            = F.softmax(outputs_, dim=2)
                log_probs        = torch.log(probs)
                log_action_probs = log_probs.gather(2, indexes_)
                policy_loss      = (-log_action_probs * advantage).sum()
                value_loss       = (.5 * (values_ - ret) ** 2.).sum()
                entropy     = (-probs * log_probs).sum()
                loss += policy_loss + 0.5 * value_loss - 0.1 * entropy
                
                self.stats["mean"]["reward"].append(reward)
                self.stats["mean"]["policy"].append(policy_loss.item())
                self.stats["mean"]["value"].append(value_loss.item())
                self.stats["mean"]["entropy"].append(entropy.item())
                self.stats["mean"]["confidence"].append(torch.exp(log_action_probs).item())
            
            if self.no_train_step % self.LOG_FREQUENCY == 0:
                msg = "{:6d}. ".format(self.no_train_step)
                msg += "  ".join("{}: {: 3.3f}".format(k, np.mean(v)) for k, v in self.stats["mean"].items())
                msg += "  " + "  ".join("{}: {:2d}".format(k, np.max(v)) for k, v in self.stats["max"].items())
                msg += "  vocab: {:3d}".format(len(self.id2word))
                print(msg)
                self.stats = {"max": defaultdict(list), "mean": defaultdict(list)}
            
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 40)
            self.optimizer.step()
            self.optimizer.zero_grad()
        
            self.transitions = []
            self.model.reset_hidden(1)
        else:
            # Keep information about transitions for Truncated Backpropagation Through Time.
            self.transitions.append([None, indexes, outputs, values])  # Reward will be set on the next call
        
        if done:
            self.last_score = 0  # Will be starting a new episode. Reset the last score.
        
        return action

cuda


<>:118: DeprecationWarning: invalid escape sequence \-


In [16]:
from collections import deque

class buffer:
    def __init__(self, max_size):
      self.max_size = max_size
      self.queue = deque([])

    def __len__(self):
      return len(self.queue)

    def __iter__(self):
      return iter(self.queue)

    def append(self, obj):
      self.queue.append(obj)
      if len(self.queue) > self.max_size:
        return self.queue.pop()
      else:
        return None

    def clear(self):
      self.queue = deque([])

# Utilities for dealing with tokens
def make_inputs(tokenizer, prompts, device="cuda"):
    token_lists = [tokenizer.encode(p) for p in prompts]
    maxlen = max(len(t) for t in token_lists)
    if "[PAD]" in tokenizer.all_special_tokens:
        pad_id = tokenizer.all_special_ids[tokenizer.all_special_tokens.index("[PAD]")]
    else:
        pad_id = 0
    input_ids = [[pad_id] * (maxlen - len(t)) + t for t in token_lists]
    # position_ids = [[0] * (maxlen - len(t)) + list(range(len(t))) for t in token_lists]
    attention_mask = [[0] * (maxlen - len(t)) + [1] * len(t) for t in token_lists]
    return dict(
        input_ids=torch.tensor(input_ids).to(device),
    #    position_ids=torch.tensor(position_ids).to(device),
        attention_mask=torch.tensor(attention_mask).to(device),
    )

def predict_token(mt, prompts, return_p=False):
    inp = make_inputs(mt.tokenizer, prompts)
    preds, p = predict_from_input(mt.model, inp)
    result = [mt.tokenizer.decode(c) for c in preds]
    if return_p:
        result = (result, p)
    return result


def predict_from_input(model, inp):
    out = model(**inp)["logits"]
    probs = torch.softmax(out[:, -1], dim=1)
    p, preds = torch.max(probs, dim=1)
    return preds, p

In [17]:
from transformers import top_k_top_p_filtering
from torch.nn import Identity
import torch.nn.functional as F

class NLPAgent:
    """ Hugging Face Transformer Agent """
    UPDATE_FREQUENCY = 10
    LOG_FREQUENCY = 10
    GAMMA = 0.9
    MEMORY_LEN = 3
    
    def __init__(self, model, model_ref, tokenizer, humanTurns=0) -> None:
        self._initialized = False
        self._epsiode_has_started = False
       
        self.memory = buffer(self.MEMORY_LEN)

        self.model = model
        self.model_ref = model_ref
        self.tokenizer = tokenizer

        self.humanTurns = humanTurns
        self.humanTurnsRem = self.humanTurns
        
        if model_ref is not None:
          # initialize trainer
          ppo_config = {'batch_size': self.UPDATE_FREQUENCY, 'forward_batch_size': 1}
          self.ppo_trainer = PPOTrainer(self.model, self.model_ref, self.tokenizer, **ppo_config)
          self.valueHead = self.ppo_trainer.valueHead

        if device == "cuda":
          self.model.cuda()
          self.tokenizer.cuda()
          
        self.mode = "test"

        self.clearTextWorldArt = True
    
    def train(self):
        if self.model_ref is None:
          raise NotImplementedError
        self.mode = "train"
        self.stats = {"max": defaultdict(list), "mean": defaultdict(list)}
        self.transitions = []
        # self.model.reset_hidden(1)
        self.last_score = 0
        self.no_train_step = 0
        self.clearTextWorldArt = True
        
        self.memory.clear()
    
    def test(self):
        self.clearTextWorldArt = True
        self.mode = "test"
        # self.model.reset_hidden(1)
        
    @property
    def infos_to_request(self) -> EnvInfos:
        return EnvInfos(description=True, inventory=True, admissible_commands=True,
                        won=True, lost=True)
      
    def _discount_rewards(self, last_values=None):
        returns, advantages = [], []
        if last_values is None:
            # not sure if this makes sense for when there is no next state
            _, _, _, R = self.transitions[-1]
        else:
            R = last_values.data
        for t in reversed(range(len(self.transitions))):
            rewards, _, _, values = self.transitions[t]
            R = rewards + self.GAMMA * R
            adv = R - values
            returns.append(R)
            advantages.append(adv)
            
        return returns[::-1], advantages[::-1]
    
    # fill in results from action and train if time
    def reportScore(self, score, done, infos):
        if self.mode == "train":
            reward = score - self.last_score  # Reward is the gain/loss in score.
            self.last_score = score
            if infos["won"]:
                reward += 100
            if infos["lost"]:
                reward -= 100

            self.transitions[-1][0] = reward  # Update reward information. Was initialized as none

            self.no_train_step += 1

            self.stats["max"]["score"].append(score)
            if self.no_train_step % self.UPDATE_FREQUENCY == 0:
                # get discounted returns and advantages across multiple actions. Currently not used
                returns, advantages = self._discount_rewards()

                query = []
                response = []
                rewardList = []

                for t in reversed(range(len(self.transitions))):
                  rew, prompt, action, values = self.transitions[t]

                  query.append(prompt[0])
                  response.append(action[0])
                  rewardList.append(rew)

                train_stats = self.ppo_trainer.step(query, response, rewardList)

                if self.no_train_step % self.LOG_FREQUENCY == 0:
                  print(train_stats)

                self.transitions = []
        
    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> Optional[str]:
        if self.clearTextWorldArt:
          self.clearTextWorldArt = False
          if "Welcome to TextWorld!" in obs:
              obs = obs[obs.index("Welcome to TextWorld!"):]
          elif "$$$$$$$" in obs:
              obs = obs[obs.rindex("$$$$$$$"):]
        
        # Build agent's observation: feedback + look + inventory.
        pastStates = ""
        for mem in self.memory:
          pastStates = pastStates + mem + "\n"
        admissible_commands_str = "options: "
        for adm_cmd in infos["admissible_commands"]:
            admissible_commands_str += adm_cmd + ", "
        input_ = "{}\n{}\n{}\n{}\nYou".format(obs, infos["description"], infos["inventory"], admissible_commands_str)
        prompt = pastStates + input_
        
        # grabs value of last token in action
        values = 0
        # convert text to tensor
        input_ids = self.tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to(device)
        print("prompt tokens: ", input_ids.shape)
        print(input_)

        if self.humanTurnsRem > 0:
          action = input()
          self.memory.append(input_ + action)
          self.humanTurnsRem -= 1
          return action

        new_tokens = 0
        next_token = None
        
        cache = None
        
        while new_tokens == 0 or (new_tokens < 20 and "\n" not in self.tokenizer.decode(next_token) 
                                  and next_token != self.tokenizer.eos_token):
          # run model
          with torch.no_grad():
                # get logits, only get last value
                if cache is None:
                    lmOut = self.model(input_ids, output_hidden_states=True, use_cache=True)
                else:
                    lmOut = self.model(input_ids[:,-1:], output_hidden_states=True, use_cache=True, past_key_values=cache)
                # print(dir(lmOut))
                logits, hidden_state, cache = lmOut.logits, lmOut.hidden_states[-1], lmOut.past_key_values
                
                # hidden_state = hidden_state[-1]
                values = self.valueHead(hidden_state)
                
                next_token_logits = logits[:, -1, :]
                next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=0, top_p=1)
                probs = F.softmax(next_token_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1).squeeze(1)
                input_ids = torch.cat([input_ids, next_token.unsqueeze(-1)], dim=-1)
                
                new_tokens += 1

        del cache
        del hidden_state
        
        action_tens = input_ids[:,-new_tokens:].to("cpu")
        prompt_tens = input_ids[:,:-new_tokens].to("cpu")
        
        del input_ids

        action = self.tokenizer.decode(action_tens[0,:])

        print("action")
        print(action)
        
        # only grab last token
        values = values[0,-1,0]
        print("last token value in action")
        print(values)

        self.memory.append(input_ + action)
        
        if self.mode == "train" and not done:
            self.transitions.append([None, prompt_tens, action_tens, values])  # Reward will be set on the next call
            
        if done:
            self.last_score = 0  # Will be starting a new episode. Reset the last score.
            self.memory.clear()
            self.clearTextWorldArt = True
            self.humanTurnsRem = self.humanTurns
            
        return action
        
        # moved to report score
#         if self.mode == "test":
#             # if done:
#             #     self.model.reset_hidden(1)
#             return action
        
#         if self.transitions:
#             reward = score - self.last_score  # Reward is the gain/loss in score.
#             self.last_score = score
#             if infos["won"]:
#                 reward += 100
#             if infos["lost"]:
#                 reward -= 100
                
#             self.transitions[-1][0] = reward  # Update reward information. Was initialized as none
        
#         self.no_train_step += 1
        
#         self.stats["max"]["score"].append(score)
#         if self.no_train_step % self.UPDATE_FREQUENCY == 0:
#             # get discounted returns and advantages across multiple actions. Currently not used
#             returns, advantages = self._discount_rewards(values)

#             query = []
#             response = []
#             rewardList = []

#             for t in reversed(range(len(self.transitions))):
#               rewards, prompt, action, values = self.transitions[t]

#               query.append(prompt[0])
#               response.append(action[0])
#               rewardList.append(rewards)

#             train_stats = self.ppo_trainer.step(query, response, rewardList)

#             if self.no_train_step % self.LOG_FREQUENCY == 0:
#               print(train_stats)
            
#             self.transitions = []
            
            
#         else:
#             # Keep information about transitions for ppo
            
#             self.transitions.append([None, prompt_tens, action_tens, values])  # Reward will be set on the next call
        
#         if done:
#             self.last_score = 0  # Will be starting a new episode. Reset the last score.
#             self.memory.clear()
#             self.clearTextWorldArt = True
#             self.humanTurnsRem = self.humanTurns
        
#         return action

In [18]:
if IS_COLAB:
  !pip install accelerate

In [19]:
if IS_COLAB:
    !pip install torchinfo

In [20]:
import trl

import importlib
importlib.reload(trl)
from trl.ppoValHead import PPOTrainer

from torchinfo import summary

# get models
# gpt2_model = GPT2HeadWithValueModel.from_pretrained('gpt2')
# gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained('gpt2')

model_name = 'gpt2-xl'
# model_name = 'gptj'

low_ram = True

# gpt2 and gpt2-xl
if 'gpt2' in model_name:
    from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model_ref = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
elif model_name == 'gptj':
    from transformers import GPT2Tokenizer, GPTJForCausalLM
    if low_ram:
        model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16, low_cpu_mem_usage=True)
        model_ref = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16, low_cpu_mem_usage=True)
    else:
        model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
        model_ref = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

print(model.config.torch_dtype)

model = model.to(device)
model_ref = model_ref.to(device)
# tokenizer doesn't have a to device
# tokenizer = tokenizer.to(device)

# model_name = r"gpt2-xl"

# Note that if you trace other models, you should set noise_level appropriately.
# (We use 0.03 for gpt-neox-20b and 0.025 for gpt-j-6b)
# model_name = r"EleutherAI/gpt-neox-20b"
# model_name = r"EleutherAI/gpt-j-6B"

# torch_dtype = torch.float16 if '20b' in model_name else None

# mt = ModelAndTokenizer(model_name, low_cpu_mem_usage=IS_COLAB, torch_dtype=torch_dtype)
# predict_token(mt, ['Megan Rapinoe plays the sport of',
#                    'The Space Needle is in the city of'
#                   ], return_p=True)

/usr/lib/python3/dist-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/usr/lib/python3/dist-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters
/usr/lib/python3/dist-packages/pandas/util/testing.py:28: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import pandas._libs.testing as _testing


None


In [22]:
agent = NLPAgent(model, model_ref, tokenizer, humanTurns=0)
print(agent.ppo_trainer.valueHead)
summary(model)

ValueHead(
  (summary): Linear(in_features=1600, out_features=1, bias=True)
  (activation): Identity()
  (first_dropout): Dropout(p=0.1, inplace=False)
  (last_dropout): Identity()
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


Layer (type:depth-idx)                             Param #
GPT2LMHeadModel                                    --
├─GPT2Model: 1-1                                   --
│    └─Embedding: 2-1                              80,411,200
│    └─Embedding: 2-2                              1,638,400
│    └─Dropout: 2-3                                --
│    └─ModuleList: 2-4                             --
│    │    └─GPT2Block: 3-1                         30,740,800
│    │    └─GPT2Block: 3-2                         30,740,800
│    │    └─GPT2Block: 3-3                         30,740,800
│    │    └─GPT2Block: 3-4                         30,740,800
│    │    └─GPT2Block: 3-5                         30,740,800
│    │    └─GPT2Block: 3-6                         30,740,800
│    │    └─GPT2Block: 3-7                         30,740,800
│    │    └─GPT2Block: 3-8                         30,740,800
│    │    └─GPT2Block: 3-9                         30,740,800
│    │    └─GPT2Block: 3-10                 

### Training the neural agent
Let's first evaluate the agent before training to get a sense of its initial performance.

In [23]:
agent.train()
play(agent, "./games/tw-rewardsDense_goalDetailed.z8")

['./games/tw-rewardsDense_goalDetailed.z8']
tw-v0
tw-rewardsDense_goalDetailed.z8prompt tokens:  torch.Size([1, 386])
Welcome to TextWorld! Here is your task for today. First of all, you could, like, open the chest drawer. Then, pick up the old key from the chest drawer. After picking up the old key, assure that the wooden door is unlocked. Then, ensure that the wooden door is open. After pulling open the wooden door, take a trip east. And then, make sure that the screen door is wide open. After that, make an effort to move east. With that done, attempt to take a trip south. Then, pick-up the carrot from the floor of the garden. And then, attempt to go to the north. If you can succeed at that, make an effort to take a trip west. And then, rest the carrot on the stove. Got that? Good!

-= Bedroom =-
You are in a bedroom. A standard kind of place.

You can make out a chest drawer. There's something strange about this being here, but you can't put your finger on it. You see a closed antiq

/home/ubuntu/trl/trl/core.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(v, collections.Mapping):


RuntimeError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 47.54 GiB total capacity; 44.94 GiB already allocated; 84.69 MiB free; 46.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from time import time
print("Training on 100 games")
agent.train()  # Tell the agent it should update its parameters.
starttime = time()
play(agent, "./training_games/", nb_episodes=100 * 5, verbose=False)  # Each game will be seen 5 times.
print("Trained in {:.2f} secs".format(time() - starttime))

# Save the trained agent.
import os
os.makedirs('checkpoints', exist_ok=True)
torch.save(agent, 'checkpoints/agent_trained_on_multiple_games.pt')

Unsurprisingly, the result is not much different from what the random agent can achieve since our neural agent is initialized to a random policy.

Let's train the agent for a few episodes.

In [ ]:
# You can skip this if you already downloaded the data in the prequisite section.
if False:
  from time import time
  agent = NeuralAgent()

  print("Training")
  agent.train()  # Tell the agent it should update its parameters.
  starttime = time()
  play(agent, "./games/tw-rewardsDense_goalDetailed.z8", nb_episodes=500, verbose=False)  # Dense rewards game.

  print("Trained in {:.2f} secs".format(time() - starttime))

  # Save the trained agent.
  import os
  os.makedirs('checkpoints', exist_ok=True)
  torch.save(agent, 'checkpoints/agent_trained_on_single_game.pt')

#### Testing the agent trained on a single game

In [ ]:
# We report the score and steps averaged over 10 playthroughs.
agent = torch.load('checkpoints/agent_trained_on_single_game.pt')
agent.test()
play(agent, "./games/tw-rewardsDense_goalDetailed.z8")  # Dense rewards game.

Of course, since we trained on that single simple game, it's not surprinsing the agent can achieve a high score on it. It would be more interesting to evaluate the generalization capability of the agent.

To do so, we are going to test the agent on another game drawn from the same game distribution (i.e. same world but the goal is to pick another food item). Let's generate `games/another_game.z8` with the same rewards density (`--rewards dense`) and the same goal description (`--goal detailed`), but using `--seed 1` and without the `--test` flag (to make sure the game is not part of the test set since `games/rewardsDense_goalDetailed.z8` is).

In [ ]:
!tw-make tw-simple --rewards dense --goal detailed --seed 1 --output games/tw-another_game.z8 -v -f

In [ ]:
# We report the score and steps averaged over 10 playthroughs.
play(RandomAgent(), "./games/tw-another_game.z8")
play(agent, "./games/tw-another_game.z8")

As we can see the trained agent does a bit better than the random agent. In order to improve the agent's generalization capability, we should train it on many different games drawn from the game distribution.

One could use the following command to easily generate 100 training games:

In [ ]:
# You can skip this if you already downloaded the data in the prequisite section.

! seq 1 100 | xargs -n1 -P4 tw-make tw-simple --rewards dense --goal detailed --format z8 --output training_games/ --seed

Then, we train our agent on that set of training games.

In [ ]:
# You can skip this if you already downloaded the data in the prequisite section.
if False:
  from time import time
  agent = NeuralAgent()

  print("Training on 100 games")
  agent.train()  # Tell the agent it should update its parameters.
  starttime = time()
  play(agent, "./training_games/", nb_episodes=100 * 5, verbose=False)  # Each game will be seen 5 times.
  print("Trained in {:.2f} secs".format(time() - starttime))

  # Save the trained agent.
  import os
  os.makedirs('checkpoints', exist_ok=True)
  torch.save(agent, 'checkpoints/agent_trained_on_multiple_games.pt')

#### Testing the agent trained on 100 games.

In [ ]:
agent = torch.load('checkpoints/agent_trained_on_multiple_games.pt')
agent.test()
play(agent, "./games/tw-rewardsDense_goalDetailed.z8")  # Averaged over 10 playthroughs.
play(agent, "./games/tw-another_game.z8")               # Averaged over 10 playthroughs.

Compare it to the agent trained on a single game.

In [ ]:
agent = torch.load('checkpoints/agent_trained_on_single_game.pt')
agent.test()
play(agent, "./games/tw-rewardsDense_goalDetailed.z8")  # Averaged over 10 playthroughs.
play(agent, "./games/tw-another_game.z8")               # Averaged over 10 playthroughs.

#### Evaluating the agent on a test distribution
We will generate 20 test games and evaluate the agent on them.

In [ ]:
# You can skip this if you already downloaded the games in the prequisite section.

! seq 1 20 | xargs -n1 -P4 tw-make tw-simple --rewards dense --goal detailed --test --format z8 --output testing_games/ --seed

In [ ]:
agent = torch.load('checkpoints/agent_trained_on_multiple_games.pt')
agent.test()
play(RandomAgent(), "./testing_games/", nb_episodes=20 * 10)
play(agent, "./testing_games/", nb_episodes=20 * 10)  # Averaged over 10 playthroughs for each test game.

While not being perfect, the agent manage to score more points on average compared to the random agent.

## Next steps

Here are a few possible directions one can take to improve the agent's performance.
- Adding more training games
- Changing the agent architecture
- Leveraging already trained word embeddings
- Playing more games at once (see [`textworld.gym.make_batch`](https://textworld.readthedocs.io/en/latest/textworld.gym.html#textworld.gym.utils.make_batch))


## Papers about RL applied to text-based games
* [Language Understanding for Text-based games using Deep Reinforcement Learning][narasimhan_et_al_2015]
* [Learning How Not to Act in Text-based Games][haroush_et_al_2017]
* [Deep Reinforcement Learning with a Natural Language Action Space][he_et_al_2015]
* [What can you do with a rock? Affordance extraction via word embeddings][fulda_et_al_2017]
* [Text-based adventures of the Golovin AI Agent][kostka_et_al_2017]
* [Using reinforcement learning to learn how to play text-based games][zelinka_2018]

[narasimhan_et_al_2015]: https://arxiv.org/abs/1506.08941
[haroush_et_al_2017]: https://openreview.net/pdf?id=B1-tVX1Pz
[he_et_al_2015]: https://arxiv.org/abs/1511.04636
[fulda_et_al_2017]: https://arxiv.org/abs/1703.03429
[kostka_et_al_2017]: https://arxiv.org/abs/1705.05637
[zelinka_2018]: https://arxiv.org/abs/1801.01999